In [3]:

# Default locations if parameter not passed
benchmark_data_path = "/scratch/rodrigo.freitas/remy-schedulers/tb-compare.pkl"


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from rich import print as rprint
import seaborn as sns
import os
from scipy.stats import bootstrap

benchmark_data = pickle.load(open(benchmark_data_path, "rb"))

df = pd.DataFrame(benchmark_data["dataframe"])


rprint(df.columns)
rprint(df.head())
rprint(df.tail())

Index(['name', 'iter', 'hosts', 'tb-time', 'node_count', 'image', 'tb_path',
       'full_output', 'type', 'time'],
      dtype='object')

name   iter                                            hosts  \
0  tb-compare   1000  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4   
1  tb-compare   1000  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4   
2  tb-compare  10000  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4   
3  tb-compare  10000  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4   
4  tb-compare   1000  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4   

    tb-time  node_count               image       tb_path  \
0  0.133651           1  scheduler-main.sif  tb-main-loop   
1  0.120610           1  scheduler-main.sif  tb-main-loop   
2  0.121488           1  scheduler-main.sif  tb-main-loop   
3  0.132441           1  scheduler-main.sif  tb-main-loop   
4  0.345512           2  scheduler-main.sif  tb-main-loop   

                                         full_output type      time  
0  [1749569648.954010] [sorgan-cpu1:196266:0]    ...  fft  0.603741  
1  [1749569649.552870] [sorgan-cpu1:196387:0]    ...  fft  0.584158  
2  [1749569650.127109] [sorgan-cpu1:196510:0]    ...  fft  0.571161  
3  [1749569651.585788] [sorgan-cpu1:196769:0]    ...  fft  0.583336  
4  [1749569652.174153] [sorgan-cpu1:196890:0]    ...  fft  1.039627

name   iter                                            hosts  \
19  tb-compare  10000  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4   
20  tb-compare   1000  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4   
21  tb-compare   1000  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4   
22  tb-compare  10000  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4   
23  tb-compare  10000  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4   

     tb-time  node_count           image         tb_path  \
19  0.279102           2  improv-144.sif  tb-tdg-no-loop   
20  0.516627           4  improv-144.sif  tb-tdg-no-loop   
21  0.515715           4  improv-144.sif  tb-tdg-no-loop   
22  0.539853           4  improv-144.sif  tb-tdg-no-loop   
23  0.523761           4  improv-144.sif  tb-tdg-no-loop   

                                          full_output type      time  
19  [1749569665.392422] [sorgan-cpu1:198698:0]    ...  fft  0.750294  
20  [1749569666.146034] [sorgan-cpu2:196397:0]    ...  fft  0.994239  
21  [1749569667.132688] [sorgan-cpu2:196482:0]    ...  fft  0.988961  
22  [1749569668.128595] [sorgan-cpu4:140253:0]    ...  fft  1.012594  
23  [1749569670.294362] [sorgan-cpu4:140420:0]    ...  fft  0.993215

In [5]:
sns.set_theme()

colors = [
    "#7f7f7f",  # grey
    "#621dac",  # main purple
    "#c5702d",  # orange
    "#000000",  # black,
    "#099892",  # teal
    "#ffd400",  # yellow
    "#7e57c4",  # pink/purple,
]
colors_backup = colors.copy()

In [6]:
def calculate_bootstrap_error(data):
    n_runs = benchmark_data["config"].metadata.runs
    if n_runs < 2:
        return 0
    res = bootstrap(
        (data,), np.mean, confidence_level=0.95, n_resamples=1000, method="basic"
    )
    return res.standard_error

In [7]:
bench_names = df["name"].unique()
run_metadata = benchmark_data["config"].applications

for bench in bench_names:
    bench_metadata = run_metadata[bench]
    n_runs = benchmark_data["config"].metadata.runs
    print(f"Benchmark: {bench} - Runs: {n_runs}")
    # print(bench_metadata)
    df_bench = df[df["name"] == bench]
    for idx, row in df_bench.iterrows():
        print(f"--- Row {idx} ---")
        for col, val in row.items():
            print(f"{col}: {val}")   # printing val directly never truncates
        print()




Benchmark: tb-compare - Runs: 2
--- Row 0 ---
name: tb-compare
iter: 1000
hosts: sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4
tb-time: 0.13365100000000002
node_count: 1
image: scheduler-main.sif
tb_path: tb-main-loop
full_output: [1749569648.954010] [sorgan-cpu1:196266:0]        ib_verbs.h:87   UCX  ERROR ibv_exp_query_device(mlx5_0) returned 95: Operation not supported
Running Task Benchmark
  Configuration:
    Task Graph 1:
      Time Steps: 32
      Max Width: 32
      Dependence Type: fft
      Radix: 3
      Period: 0
      Fraction Connected: 0.250000
      Kernel:
        Type: compute_bound
        Iterations: 1000
        Samples: 16
        Imbalance: 0.000000
      Output Bytes: 16
      Scratch Bytes: 0
Loop 0 : Elapsed time : 0.036255
Loop 1 : Elapsed time : 0.022455
Loop 2 : Elapsed time : 0.023047
Loop 3 : Elapsed time : 0.022934
Loop 4 : Elapsed time : 0.022915
Total Tasks 1024
Total Dependencies 2666
Total FLOPs 131137536
Total Bytes 0
Elapsed Time 1.336510e-01 sec